In [ ]:
"""Author-Abhishek Darana - 07/18/2024 - I have tried building a model using detectron2. Built-in evaluation metrics are failing due single point masks have to fix that"""

In [1]:
import os
import json
import cv2
import random
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.utils.visualizer import Visualizer, ColorMode

In [2]:
# Custom function to load COCO formatted dataset with absolute paths
def load_coco_json(json_file):
    with open(json_file) as f:
        coco_dict = json.load(f)

    dataset_dicts = []
    for image_info in coco_dict["images"]:
        record = {}
        record["file_name"] = image_info["file_name"]  # Absolute path to the image
        record["image_id"] = image_info["id"]
        record["height"] = image_info["height"]
        record["width"] = image_info["width"]

        annos = [anno for anno in coco_dict["annotations"] if anno["image_id"] == image_info["id"]]
        objs = []
        for anno in annos:
            obj = {
                "bbox": anno["bbox"],
                "bbox_mode": BoxMode.XYWH_ABS,
                "category_id": anno["category_id"],
                "segmentation": anno.get("segmentation"),
                "iscrowd": anno.get("iscrowd", 0)
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    
    return dataset_dicts

# Paths to your JSON file
json_file = os.path.join("C:\\Users\\Abhishek\\Documents\\Summer_research\\CHARISMA-main\\ground-penetrating-radar\\Rebar mapping\\code\\COCO_scripts\\coco_dataset_0.json")#"C:\\Users\\Abhishek\\Documents\\Summer_research\\CHARISMA-main\\ground-penetrating-radar\\Rebar mapping\\code\\coco_dataset.json")

dataset_dicts = load_coco_json(json_file)

In [3]:
random.shuffle(dataset_dicts)
split_index = int(0.8 * len(dataset_dicts))  # 80% for training, 20% for validation
train_dataset_dicts = dataset_dicts[:split_index]
val_dataset_dicts = dataset_dicts[split_index:]

# Register the dataset
def register_dataset(dataset_dicts, name):
    DatasetCatalog.register(name, lambda: dataset_dicts)
    MetadataCatalog.get(name).set(thing_classes=["Rebar"])

register_dataset(train_dataset_dicts, "my_dataset_train")
register_dataset(val_dataset_dicts, "my_dataset_val")

In [4]:
# Setup configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
# cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.DEVICE = "cpu"
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2  # Batch size

# Calculate the number of iterations
NUM_TRAIN_IMAGES = len(load_coco_json(json_file))  # Number of training images
BATCH_SIZE = cfg.SOLVER.IMS_PER_BATCH
NUM_EPOCHS = 20  # Number of epochs you want to train

# Calculate the number of iterations per epoch
iters_per_epoch = NUM_TRAIN_IMAGES // BATCH_SIZE

# Set the maximum number of iterations
cfg.SOLVER.MAX_ITER = NUM_EPOCHS * iters_per_epoch

cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.STEPS = []  # Optionally, add custom learning rate steps
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Set to the number of classes in your dataset

# Output directory for saving checkpoints and logs
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[07/17 19:07:59 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[07/17 19:07:59 d2.engine.train_loop]: Starting training from iteration 0


c:\Users\Abhishek\anaconda3\envs\detectron2_env\Lib\site-packages\torch\functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[07/17 19:13:32 d2.utils.events]:  eta: 0:37:27  iter: 19  total_loss: 0.648  loss_cls: 0.6446  loss_box_reg: 0  loss_rpn_cls: 0.02089  loss_rpn_loc: 0    time: 16.1605  last_time: 16.4508  data_time: 0.3325  last_data_time: 0.0038   lr: 2.9908e-05  
[07/17 19:19:08 d2.utils.events]:  eta: 0:33:05  iter: 39  total_loss: 0.2757  loss_cls: 0.2483  loss_box_reg: 0  loss_rpn_cls: 0.02001  loss_rpn_loc: 0    time: 16.4743  last_time: 18.9550  data_time: 0.0038  last_data_time: 0.0041   lr: 6.1127e-05  
[07/17 19:24:39 d2.utils.events]:  eta: 0:27:34  iter: 59  total_loss: 0.06598  loss_cls: 0.05423  loss_box_reg: 0  loss_rpn_cls: 0.01043  loss_rpn_loc: 0    time: 16.5123  last_time: 14.5268  data_time: 0.0037  last_data_time: 0.0030   lr: 9.2345e-05  
[07/17 19:30:23 d2.utils.events]:  eta: 0:22:19  iter: 79  total_loss: 0.0215  loss_cls: 0.01885  loss_box_reg: 0  loss_rpn_cls: 0.0001484  loss_rpn_loc: 0    time: 16.6867  last_time: 14.7854  data_time: 0.0036  last_data_time: 0.0032   lr: 0

In [11]:
from detectron2.evaluation import DatasetEvaluator

class TwoCoordinateEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        self.dataset_name = dataset_name
        self.predictions = []

    def reset(self):
        self.predictions = []

    def process(self, inputs, outputs):
        for instances in outputs:
            assert isinstance(instances, Instances), "Instances should be of type Instances"

            # Assuming pred_points is a tensor, convert it to numpy array
            pred_points = instances.pred_points.cpu().numpy()  # Example: Access pred_points
            ground_truth_points = inputs["points"]  # Example: Access ground truth points

            self.predictions.append((ground_truth_points, pred_points))

    def evaluate(self):
        # Calculate metrics based on predictions and ground truth points
        metrics = {}
        for gt_points, pred_points in self.predictions:
            # Example: Calculate distance metrics or accuracy
            distances = np.linalg.norm(gt_points - pred_points, axis=1)
            mean_distance = np.mean(distances)
            metrics['mean_distance'] = mean_distance
        return metrics

# Setup evaluator
evaluator = TwoCoordinateEvaluator("my_dataset_val")

# Perform evaluation
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
inference_on_dataset(trainer.model, val_loader, evaluator)

# Get evaluation results
metrics = evaluator.evaluate()
print(metrics)

[07/17 21:06:47 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[07/17 21:06:47 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[07/17 21:06:47 d2.data.common]: Serializing 4 elements to byte tensors and concatenating them all ...
[07/17 21:06:47 d2.data.common]: Serialized dataset takes 0.00 MiB
[07/17 21:06:47 d2.evaluation.evaluator]: Start inference on 4 batches


NameError: name 'Instances' is not defined

In [5]:
# Evaluation
evaluator = COCOEvaluator("my_dataset_val", cfg, False, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
inference_on_dataset(trainer.model, val_loader, evaluator)

WARNING [07/17 19:53:37 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[07/17 19:53:37 d2.evaluation.coco_evaluation]: Trying to convert 'my_dataset_val' to COCO format ...
[07/17 19:53:37 d2.data.datasets.coco]: Converting annotations of dataset 'my_dataset_val' to COCO format ...)
[07/17 19:53:37 d2.data.datasets.coco]: Converting dataset dicts into COCO format


ValueError: Cannot create a polygon from 2 coordinates.

In [ ]:
# Visualization
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
# Load metadata
metadata = MetadataCatalog.get("my_dataset_val")

In [ ]:
# Visualize a few test images
for d in random.sample(load_coco_json(json_file), 3):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], metadata=metadata, scale=0.8, instance_mode=ColorMode.IMAGE_BW)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("Prediction", v.get_image()[:, :, ::-1])
    cv2.waitKey(0)
cv2.destroyAllWindows()